In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib as plt
from tqdm import tqdm, trange
import torch
import pickle
import io

In [ ]:
pip install --upgrade scipy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/"stats 209 rawr"

/content/drive/.shortcut-targets-by-id/1CvZNtlac_E1PnMgOazeQEBGEv0dIGoLe/stats 209 rawr


In [ ]:
import numpy as np
from scipy.io import mmread
from scipy.sparse import csr_matrix

#fh = io.BytesIO()

def mtx_to_adjacency_matrix(file_path):
    # Read the Matrix Market file
    matrix = mmread(file_path)

    # Convert the matrix to a compressed sparse row (CSR) format
    matrix_csr = csr_matrix(matrix)
    print('nonzeros:', matrix_csr.nnz)

    # Convert the CSR matrix to a dense array (adjacency matrix)
    adjacency_matrix = matrix_csr.toarray()

    return adjacency_matrix

# Example usage
file_path = 'bio-diseasome.mtx'  # Replace with the actual path to your .mtx file
A = mtx_to_adjacency_matrix(file_path)
vertices = np.shape(A)[0] # 516
# a = mmread('bio-diseasome.mtx')
# graph = nx.Graph(a)

#print("Adjacency Matrix:")
# print(adjacency_matrix)

nonzeros: 2376


In [ ]:
# edges_df = pd.read_csv('bio-grid-mouse.edges', header=None, delimiter=' ', names=['Node1', 'Node2'])

# # Create a graph from the edges DataFrame
# G = nx.from_pandas_edgelist(edges_df, 'Node1', 'Node2')

# # Create an adjacency matrix
# A = nx.adjacency_matrix(G).todense()
# vertices = len(A[0])

In [ ]:
device = torch.device("cuda")

In [ ]:
#second degree neighbor matrix
A_gpu = torch.tensor(A).float().to(device)
A_2_gpu = torch.matmul(A_gpu, A_gpu)
A_2 = A_2_gpu.cpu().numpy()
A_2 = np.where(A_2 > 0, 1, 0)
np.fill_diagonal(A_2, 0)
A_2 = A_2 - A # second degree neighbors
A_2_gpu = torch.tensor(A_2).float().to(device)

In [ ]:
#set parameters
#vertices = 1000
#adjacencies = 100
# pi_vals = [0.25, 0.5, 0.75]
pi_vals = [0.25, 0.5, 0.75]
# num_trials = 1000

In [ ]:
replicate = {}

In [ ]:
#beta 1 is treatment assignment
#beta 2 is neighbors
#beta 3 is second degree neighbors
def add_test_to_dictionary(beta_1, beta_2, beta_3, replicate):
  print(f'\nBETAS {beta_1}, {beta_2}, {beta_3}\n')
  for pi in pi_vals:
    ADE = []
    AIE_1 = []
    AIE_2 = []
    for i in trange(1000):
      # treatment assignment
      W = np.random.binomial(1, pi, vertices)
      W_gpu = torch.tensor(W).float().to(device)

      # first deg effect
      if beta_2 != 0:
        W_A = torch.matmul(W_gpu, A_gpu).cpu().numpy()
      else:
        W_A = np.zeros(vertices)

      # second deg effect
      if beta_3 != 0:
        W_A_2 = torch.matmul(W_gpu, A_2_gpu).cpu().numpy()
      else:
        W_A_2 = np.zeros(vertices)

      # noise
      noise = np.random.normal(0, 1, vertices)

      # generate Y
      s1 = np.where(np.sum(A, axis=1) == 0, 0.000001, np.sum(A, axis=1)) # circumvent div by 0 errors
      # A_1_frac = np.where(np.sum(A, axis=1) != 0, np.divide(W_A, np.sum(A, axis=1, keepdims=True)), 0)
      A_1_frac = np.where(W_A == 0, 0, np.divide(W_A, s1))
      s2 = np.where(np.sum(A_2, axis=1) == 0, 0.000001, np.sum(A_2, axis=1)) # circumvent div by 0 errors
      # A_2_frac = np.where(np.logical_and(np.sum(A_2, axis=1) != 0, W_A_2 != 0), np.divide(W_A_2, np.sum(A_2, axis=1, keepdims=True)), 0)
      A_2_frac = np.where(W_A_2 == 0, 0, np.divide(W_A_2, s2)) # W_A_2 == 0 --> A_2 == 0 (there are more W_A_2 = 0s)
      Y = A_2_frac * beta_3 + A_1_frac * beta_2 + W * (beta_1) + noise

      # ADE
      average_direct_effect = ((np.dot(Y,W))/(pi) - (np.dot(Y, (1 - W)))/(1-pi)) / vertices
      ADE.append(average_direct_effect)

      # AIE
      # E = first degree
      average_indirect_effect1 = np.dot(Y, ((W_A) / pi - ((1 - W) @ A) / (1 - pi))) / vertices
      average_indirect_effect2 = np.dot(Y, ((W_A_2) / pi - ((1 - W) @ A_2) / (1 - pi))) / vertices
      AIE_1.append(average_indirect_effect1)
      AIE_2.append(average_indirect_effect2)

    str1 = str(pi)+" ADE_{}_{}_{}".format(round(beta_1, 2), round(beta_2, 2), round(beta_3, 2))
    str2 = str(pi)+" AIE_1_{}_{}_{}".format(round(beta_1, 2), round(beta_2, 2), round(beta_3, 2))
    str3 = str(pi)+" AIE_2_{}_{}_{}".format(round(beta_1, 2), round(beta_2, 2), round(beta_3, 2))
    replicate[str1] = ADE
    replicate[str2] = AIE_1
    replicate[str3] = AIE_2

    with open('equation_1_biodiseasome_batch.pkl', 'wb') as fp:
      pickle.dump(replicate, fp)

In [ ]:
betas = []
# baseline where no interference
betas.append((1, 0, 0))

#only first degree neighbors
betas.append((1/3, 2/3, 0))
betas.append((0.5, 0.5, 0))
betas.append((2/3, 1/3, 0))

#only second degree neighbors
betas.append((1/3, 0, 2/3))
betas.append((0.5, 0, 0.5))
betas.append((2/3, 0, 1/3))

#first and second degree neighbors
betas.append((1/3, 5/9, 1/9))
betas.append((1/3, 1/3, 1/3))
betas.append((1/3, 1/9, 5/9))

In [ ]:
for beta in betas:
  add_test_to_dictionary(beta[0], beta[1], beta[2], replicate)


BETAS 1, 0, 0



100%|██████████| 1000/1000 [00:01<00:00, 734.88it/s]



BETAS 0.3333333333333333, 0.6666666666666666, 0



100%|██████████| 1000/1000 [00:01<00:00, 695.96it/s]



BETAS 0.5, 0.5, 0



...

100%|██████████| 1000/1000 [00:01<00:00, 645.66it/s]



BETAS 0.3333333333333333, 0.3333333333333333, 0.3333333333333333



100%|██████████| 1000/1000 [00:01<00:00, 614.38it/s]



BETAS 0.3333333333333333, 0.1111111111111111, 0.5555555555555556



100%|██████████| 1000/1000 [00:01<00:00, 656.03it/s]


In [ ]:
print(replicate.keys())

dict_keys(['0.25 ADE_0.33_0_0.67', '0.25 AIE_1_0.33_0_0.67', '0.25 AIE_2_0.33_0_0.67', '0.25 ADE_0.33_0.33_0.33', '0.25 AIE_1_0.33_0.33_0.33', '0.25 AIE_2_0.33_0.33_0.33', '0.25 ADE_1_0_0', '0.25 AIE_1_1_0_0', '0.25 AIE_2_1_0_0', '0.5 ADE_1_0_0', '0.5 AIE_1_1_0_0', '0.5 AIE_2_1_0_0', '0.75 ADE_1_0_0', '0.75 AIE_1_1_0_0', '0.75 AIE_2_1_0_0', '0.5 ADE_0.33_0_0.67', '0.5 AIE_1_0.33_0_0.67', '0.5 AIE_2_0.33_0_0.67', '0.75 ADE_0.33_0_0.67', '0.75 AIE_1_0.33_0_0.67', '0.75 AIE_2_0.33_0_0.67', '0.5 ADE_0.33_0.33_0.33', '0.5 AIE_1_0.33_0.33_0.33', '0.5 AIE_2_0.33_0.33_0.33', '0.75 ADE_0.33_0.33_0.33', '0.75 AIE_1_0.33_0.33_0.33', '0.75 AIE_2_0.33_0.33_0.33', '0.25 ADE_0.33_0.67_0', '0.25 AIE_1_0.33_0.67_0', '0.25 AIE_2_0.33_0.67_0', '0.5 ADE_0.33_0.67_0', '0.5 AIE_1_0.33_0.67_0', '0.5 AIE_2_0.33_0.67_0', '0.75 ADE_0.33_0.67_0', '0.75 AIE_1_0.33_0.67_0', '0.75 AIE_2_0.33_0.67_0', '0.25 ADE_0.5_0.5_0', '0.25 AIE_1_0.5_0.5_0', '0.25 AIE_2_0.5_0.5_0', '0.5 ADE_0.5_0.5_0', '0.5 AIE_1_0.5_0.5_0', '0

In [ ]:
df_r = pd.DataFrame.from_dict(replicate)
df_r.head()

,0.25 ADE_0.33_0_0.67,0.25 AIE_1_0.33_0_0.67,0.25 AIE_2_0.33_0_0.67,0.25 ADE_0.33_0.33_0.33,0.25 AIE_1_0.33_0.33_0.33,0.25 AIE_2_0.33_0.33_0.33,0.25 ADE_1_0_0,0.25 AIE_1_1_0_0,0.25 AIE_2_1_0_0,0.5 ADE_1_0_0,...,0.75 AIE_2_0.33_0.56_0.11,0.25 ADE_0.33_0.11_0.56,0.25 AIE_1_0.33_0.11_0.56,0.25 AIE_2_0.33_0.11_0.56,0.5 ADE_0.33_0.11_0.56,0.5 AIE_1_0.33_0.11_0.56,0.5 AIE_2_0.33_0.11_0.56,0.75 ADE_0.33_0.11_0.56,0.75 AIE_1_0.33_0.11_0.56,0.75 AIE_2_0.33_0.11_0.56
0,0.579583,-0.654626,-0.192046,0.385683,0.559184,0.517791,1.308748,-1.224282,-5.776654,0.843289,...,-1.095516,0.204656,0.003351,-0.126543,0.331498,0.141006,-0.189471,0.309062,0.285369,0.424698
1,0.343758,-0.914776,0.499797,0.317578,0.635929,0.636901,0.935474,-1.784548,-6.515429,0.964231,...,2.263811,0.546913,-0.192694,1.065505,0.142052,0.175665,0.850705,0.375782,0.458506,3.975438
2,0.374293,-1.261133,0.979140,0.264304,0.495010,2.064999,1.066546,-0.860742,-4.269297,1.112542,...,4.631519,0.262865,0.536137,0.513195,0.432161,0.262026,-0.526909,-0.171014,-0.797933,-2.657969
3,0.310644,-1.392636,1.482475,0.400181,0.120870,1.735997,0.825486,-1.773568,-5.663454,0.980868,...,1.107124,0.348404,0.441031,1.492699,0.323176,0.796978,0.311274,0.468299,0.862223,0.871014
4,0.358134,-0.679166,1.173598,0.296258,0.573377,-0.248802,0.888606,-0.923374,-2.976014,0.898795,...,-0.566744,0.309011,-0.440770,0.276832,0.318485,0.316046,2.096831,0.313541,0.609972,2.058994


In [ ]:
df_r.mean()

0.25 ADE_0.33_0_0.67         0.334851
0.25 AIE_1_0.33_0_0.67      -1.152974
0.25 AIE_2_0.33_0_0.67       0.775556
0.25 ADE_0.33_0.33_0.33      0.334572
0.25 AIE_1_0.33_0.33_0.33    0.303392
                               ...   
0.5 AIE_1_0.33_0.11_0.56     0.070680
0.5 AIE_2_0.33_0.11_0.56     0.600060
0.75 ADE_0.33_0.11_0.56      0.335528
0.75 AIE_1_0.33_0.11_0.56    0.116313
0.75 AIE_2_0.33_0.11_0.56    0.759457
Length: 90, dtype: float64

In [ ]:
df_r.to_csv('equation_1_biodiseasome.csv')

In [ ]:
dfnew = pd.read_csv('equation_1_biodiseasome.csv')
dfnew.head()

,Unnamed: 0,0.25 ADE_0.33_0_0.67,0.25 AIE_1_0.33_0_0.67,0.25 AIE_2_0.33_0_0.67,0.25 ADE_0.33_0.33_0.33,0.25 AIE_1_0.33_0.33_0.33,0.25 AIE_2_0.33_0.33_0.33,0.25 ADE_1_0_0,0.25 AIE_1_1_0_0,0.25 AIE_2_1_0_0,...,0.75 AIE_2_0.33_0.56_0.11,0.25 ADE_0.33_0.11_0.56,0.25 AIE_1_0.33_0.11_0.56,0.25 AIE_2_0.33_0.11_0.56,0.5 ADE_0.33_0.11_0.56,0.5 AIE_1_0.33_0.11_0.56,0.5 AIE_2_0.33_0.11_0.56,0.75 ADE_0.33_0.11_0.56,0.75 AIE_1_0.33_0.11_0.56,0.75 AIE_2_0.33_0.11_0.56
0,0,0.579583,-0.654626,-0.192046,0.385683,0.559184,0.517791,1.308748,-1.224282,-5.776654,...,-1.095516,0.204656,0.003351,-0.126543,0.331498,0.141006,-0.189471,0.309062,0.285369,0.424698
1,1,0.343758,-0.914776,0.499797,0.317578,0.635929,0.636901,0.935474,-1.784548,-6.515429,...,2.263811,0.546913,-0.192694,1.065505,0.142052,0.175665,0.850705,0.375782,0.458506,3.975438
2,2,0.374293,-1.261133,0.979140,0.264304,0.495010,2.064999,1.066546,-0.860742,-4.269297,...,4.631519,0.262865,0.536137,0.513195,0.432161,0.262026,-0.526909,-0.171014,-0.797933,-2.657969
3,3,0.310644,-1.392636,1.482475,0.400181,0.120870,1.735997,0.825486,-1.773568,-5.663454,...,1.107124,0.348404,0.441031,1.492699,0.323176,0.796978,0.311274,0.468299,0.862223,0.871014
4,4,0.358134,-0.679166,1.173598,0.296258,0.573377,-0.248802,0.888606,-0.923374,-2.976014,...,-0.566744,0.309011,-0.440770,0.276832,0.318485,0.316046,2.096831,0.313541,0.609972,2.058994
